# 2장. 다항 선형 회귀

---

## 학습 목표
- 다항 선형 회귀 모델의 구조와 고차항이 추가될수록 변화하는 특성을 학습합니다. 
- 다항 선형 회귀 class를 구현합니다.

---

## 목차

### 1. 다항 선형 회귀 모델
1. 다항 선형 회귀란?

### 2. 이차항 선형 회귀
1. 이차항 선형 회귀 모델
2. 이차항 선형 회귀 class 구조
3. scikit-learn 모듈

### 3. 고차항 선형 회귀
1. 고차항 선형 회귀 모델
2. 고차항 선형 회귀 class 구조
3. scikit-learn 모듈


---

## 3.  고차항 선형 회귀

### 3-1. 고차항 선형 회귀 모델

#### 학습 데이터

이차항 선형 회귀 모델을 잘 이해했다면 고차항 선형 회귀는 그 확장형이라 생각할 수 있습니다.

이차항 선형 회귀 모델에서와 같이 데이터부터 정의해 봅시다.

##### 학습용 feature 데이터

> $$X_{feature}=\begin{pmatrix}
1 & x_{1,1} & ... & x_{1,p} \\ 
1 & x_{2,1} & ... & x_{2,p} \\ 
1 & \vdots  & \ddots  & \vdots \\ 
1 & x_{N,1} & ... & x_{N,p}
\end{pmatrix}, \;\;\;$$

##### 학습용 lable 데이터

> $$\mathbf{y}=\begin{pmatrix}
y_1 \\ 
y_2 \\ 
\vdots \\ 
y_N
\end{pmatrix}. \;\;\;\;\; $$

다중 선형 회귀와 마찬가지로 $p$개의 feature column을 갖는 feature 데이터로 일반화하여 학습데이터를 정의합니다.

#### 고차항 선형 회귀 모델

다음으로 고차항 선형 모델에 대해서 정의해 봅시다.

##### 고차항 선형 회귀 모델

> $$F(X) = X\mathbf{w}$$

> $$X=\begin{pmatrix}
1 & x_{1,1} & \cdots & x_{1,1}^{q} & x_{1,2} & \cdots & x_{1,2}^{q} & \cdots & x_{1,p}^{q} \\ 
1 & x_{2,1} & \cdots & x_{2,1}^{q} & x_{2,2} & \cdots & x_{2,2}^{2} & \cdots & x_{2,p}^{q} \\ 
\vdots & \vdots  & \cdots & \vdots & \vdots & \cdots & \vdots & \vdots & \vdots \\ 
1 & x_{N,1} & \cdots & x_{N,1}^{q} & x_{N,2} & \cdots & x_{N,2}^{q} & \cdots & x_{N,p}^{q}
\end{pmatrix}, \;\;\;\;\; \mathbf{w}=\begin{pmatrix}
w_0 \\ 
w_1 \\ 
w_2 \\
\vdots \\
w_{pq} \\
\end{pmatrix}, \;\;\;\;\; $$

$X$는 기존 feature 데이터에서 1만으로 이루어진 column과 $p$번째 feature$q$차항까지의 $x_{i}^{q}$ 데이터의 column들을 추가한 형태를 의미합니다.

##### <예제 1> 고차항 선형 회귀 모델

1개의 column 벡터를 가지는 feature 데이터를 입력하였을 때, 3차항으로 모델을 구현하고 그 예측값을 출력하여 봅시다.

In [59]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# feature 데이터
X_feature = np.array([1, 2, 3, 4]).reshape((-1,1))

# 3차항 선형 회귀 X 선언
X = np.c_[np.ones((X_feature.shape[0],1)),X_feature]
for i in range(2):
    X = np.c_[X,X_feature**(i+2)]

# label 데이터
Y = np.array([3.1, 4.9, 7.2, 8.9]).reshape((-1,1))

print("X_feature: \n{}\n".format(X_feature))
print("X: \n{}\n".format(X))
print("Y: \n{}\n".format(Y))

X_feature: 
[[1]
 [2]
 [3]
 [4]]

X: 
[[ 1.  1.  1.  1.]
 [ 1.  2.  4.  8.]
 [ 1.  3.  9. 27.]
 [ 1.  4. 16. 64.]]

Y: 
[[3.1]
 [4.9]
 [7.2]
 [8.9]]



In [60]:
# 파라미터 column 설정
w = np.ones((X.shape[1],1))

# 다항 선형 모델 함수
def F_X(w,X):
    return np.dot(X,w)

print("F(X): \n{}\n".format(F_X(w,X)))

F(X): 
[[ 4.]
 [15.]
 [40.]
 [85.]]



#### loss 함수 최적화

다음 loss 함수는 다중 선형 회귀와 똑같은 형태로 정리됩니다.

##### loss 함수

> $$
\begin{aligned}
Loss(\mathbf{w})&=\frac{1}{N}\sum_{i=1}^{N}(y_i-f(\mathbf{x_i}))^{2} \\
&=\frac{1}{N}(\mathbf{y}-X\mathbf{w})^{T}(\mathbf{y}-X\mathbf{w})
\end{aligned}$$

결과적으로 least square solution 또한 같습니다.

##### 고차항 선형 회귀 파라미터 least square solution

> $$\mathbf{\widehat{w}}=(X^{T}X)^{-1}X^{T}\mathbf{y}$$

**least square 해의 존재 문제** 

$X$는 N x (qp+1) 행렬이기에 $(X^{T}X)$는 (qp+1) x (qp+1) 행렬입니다.

그런데 만약 샘플의 수 N이 (qp+1)보다 작게 된다면 $X$의 full rank의 값이 $N$이 되기에 온전한 해를 얻을 수 없습니다.

따라서 위의 `<예제 1>`에서의 4개의 샘플에서 $p=1,q=3$이기에 $qp+1=4$이기에 $q$를 늘리는 순간 least square solution은 의미가 없습니다.

하지만 일반적으로 샘플 수 $N$은 매우 크기에 큰 문제가 되지는 않습니다.

##### <예제 2> 고차항 선형 회귀 구현 - least square

`<예제 1>`에 이어서 고차항 선형 회귀 최적의 파라미터를 least square로 구하고 최소화 된 loss 값을 출력해 봅시다.

In [1]:
import numpy as np


# feature 데이터
X_feature = np.array([1, 2, 3, 4]).reshape((-1,1))

# 3차항 선형 회귀 X 선언
X = np.c_[np.ones((X_feature.shape[0],1)),X_feature]
for i in range(2):
    X = np.c_[X,X_feature**(i+2)]

# label 데이터
Y = np.array([3.1, 4.9, 7.2, 8.9]).reshape((-1,1))


w = np.dot(np.linalg.inv(np.dot(X.transpose(),X)),np.dot(X.transpose(),Y))

# 다중 선형 모델 함수
def F_X(w,X):
    return np.dot(X,w)


def loss(f_x, label_data):
    error = label_data - f_x
    ls = np.mean(error**2)
    return ls

print("loss: {}\n".format(loss(F_X(w,X),Y)))

loss: 1.545584617537555e-22



### 2-2. 고차항 선형 회귀 class 구조

위에서 수행된 것을 확인해보면 고차항 선형 회귀의 class는 $X$를 만들어주는 것 이외에는 다중 선형 회귀의 class와 다를게 없습니다.

##### <예제 3> 고차항 선형 회귀 class

class는 least square를 사용했던 단순 선형 회귀 구조와 비슷하게 초기화, 학습, 예측, loss 함수로 구성합니다.

In [61]:
import numpy as np

class poly_linaer_regression:
    # 초기화 함수
    def __init__(self, initial_w, d):
        self.w = initial_w
        # degree는 고차항의 수를 의미
        self.degree = d
        
    # 학습 함수    
    def fit(self, feature, label):
        
        # 고차항을 추가하여 X를 만드는 과정
        X = np.c_[np.ones((feature.shape[0],1)),feature]
        for i in range(self.degree-1):
            X = np.c_[X,feature**(i+2)]
        self.w = np.dot(np.linalg.inv(np.dot(X.transpose(),X)),np.dot(X.transpose(),label))

        
    # 예측값 계산 함수
    def predict(self, feature):
        
        # 고차항을 추가하여 X를 만드는 과정
        X = np.c_[np.ones((feature.shape[0],1)),feature]
        for i in range(self.degree-1):
            X = np.c_[X,feature**(i+2)]
        prediction = np.dot(X,self.w)
        
        return prediction
    
    # loss 값 계산 함수
    def loss(self, feature, label):
        
        error = label - self.predict(feature)
        ls = np.mean(error**2)
        
        return ls

##### <예제 4> 고차항 선형 회귀 학습

`<예제 3>` 고차항 선형 회귀 class에서 구현한 class를 사용하여 학습 과정을 수행해 봅시다.

In [62]:
import numpy as np

# 학습용 데이터
feature_data = np.array([1, 2, 3, 4]).reshape((-1,1))
label_data = np.array([3.1,4.9,7.2,8.9]).reshape((-1,1))

# 파라미터 0으로 초기화
w = np.zeros((feature_data.shape[1]+1,1))
            
# 다중 선형 회귀 모델 불러오기 및 초기화
model = poly_linaer_regression(w,3)

# 학습 수행
model.fit(feature_data, label_data)

print("loss: {}\n".format(model.loss(feature_data, label_data))) 

loss: 1.545584617537555e-22



### 2-3. scikit-learn 모듈

고차항 선형회귀와 동일한 방식으로 진행합니다.

##### <예제 5> `PolynomialFeatures` 사용하기

고차항 선형 회귀를 scikit-learn으로 수행하기 위해서 `PolynomialFeatures`을 사용하여 feature 데이터의 3차항 column 배열을 추가해 봅시다.

In [63]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

feature_data = np.array([1, 2, 3, 4]).reshape((-1,1))
print("feature data: \n{}\n".format(feature_data))

# 3차항 선형 회귀에 해당되는 3를 초기값으로 입력
poly = PolynomialFeatures(3)

# fit 함수를 사용하여 X로 변환 작업을 수행
X = poly.fit_transform(feature_data)
print("X: \n{}\n".format(X))


feature data: 
[[1]
 [2]
 [3]
 [4]]

X: 
[[ 1.  1.  1.  1.]
 [ 1.  2.  4.  8.]
 [ 1.  3.  9. 27.]
 [ 1.  4. 16. 64.]]



`PolynomialFeatures`을 사용하여 $X$가 준비되었다면 단순 선형 회귀와 같은 방법으로 학습을 수행할 수 있습니다.

##### <예제 6>  scikit-learn을 사용한 고차항 선형 회귀

`<예제 4>` 에서와 같은 데이터를 사용하여 이번엔 scikit-learn의 loss 값을 출력해 봅시다.

In [64]:
import numpy as np
from sklearn.linear_model import LinearRegression

# 모델 설정
sci_model = LinearRegression()

# 학습 수행
sci_model.fit(X, label_data)

# scikit-learn 에서는 loss 함수가 모델안에 내정되어 있지 않기에 정의
def loss(prediction, label):
        
    error = label - prediction
    ls = np.mean(error**2)

    return ls

print("loss: {}\n".format(loss(sci_model.predict(X), label_data))) 

loss: 5.17689969051289e-30



---